# t-prompts Demo

This notebook demonstrates the `t-prompts` library, which provides provenance-preserving prompts using Python 3.14's template strings (t-strings).

**Requirements**: Python 3.14+

## 1. Setup and Basic Usage

In [1]:
from t_prompts import prompt

# Create a simple prompt with a labeled interpolation
instructions = "Always answer politely."
p = prompt(t"Obey {instructions:inst}")

# Renders like an f-string
print("Rendered:", str(p))
print()

# But preserves full provenance
node = p['inst']
print(f"Key: {node.key}")
print(f"Expression: {node.expression}")
print(f"Value: {node.value}")
print(f"Format spec: {node.format_spec}")

Rendered: Obey Always answer politely.

Key: inst
Expression: instructions
Value: Always answer politely.
Format spec: inst


## 2. Keying Rules

Structured prompts use either the format spec or the expression as the dictionary key:

In [2]:
x = "X"

# With format spec: key comes from format spec
p1 = prompt(t"{x:custom_key}")
print("With format spec:")
print(f"  Keys: {list(p1.keys())}")
print(f"  Value at 'custom_key': {p1['custom_key'].value}")
print()

# Without format spec: key comes from expression
p2 = prompt(t"{x}")
print("Without format spec:")
print(f"  Keys: {list(p2.keys())}")
print(f"  Value at 'x': {p2['x'].value}")

With format spec:
  Keys: ['custom_key']
  Value at 'custom_key': X

Without format spec:
  Keys: ['x']
  Value at 'x': X


## 3. Conversions

T-strings support conversion flags (!s, !r, !a) which are applied during rendering:

In [3]:
text = "hello\nworld"

# !s converts to string (calls str())
p_s = prompt(t"Text: {text!s:data}")
print(f"!s: {str(p_s)}")

# !r converts to repr (calls repr())
p_r = prompt(t"Text: {text!r:data}")
print(f"!r: {str(p_r)}")

# !a converts to ascii repr (calls ascii())
p_a = prompt(t"Text: {text!a:data}")
print(f"!a: {str(p_a)}")
print()

# Check conversion metadata
print(f"Conversion flag in !r version: {p_r['data'].conversion}")

!s: Text: hello
world
!r: Text: 'hello\nworld'
!a: Text: 'hello\nworld'

Conversion flag in !r version: r


## 4. Composing Prompts

The real power: prompts can contain other prompts, creating navigable trees:

In [4]:
# Build prompts from smaller pieces
system_msg = "You are a helpful assistant."
user_query = "What is Python?"

p_system = prompt(t"{system_msg:system}")
p_user = prompt(t"User: {user_query:query}")

# Compose into larger prompt
p_full = prompt(t"{p_system:sys}\n\n{p_user:usr}")

print("Full prompt:")
print(str(p_full))
print()

# Navigate the tree
print("Navigation:")
print(f"  p_full['sys'] type: {type(p_full['sys'].value).__name__}")
print(f"  p_full['sys']['system'].value: {p_full['sys']['system'].value}")
print(f"  p_full['usr']['query'].value: {p_full['usr']['query'].value}")

Full prompt:
You are a helpful assistant.

User: What is Python?

Navigation:
  p_full['sys'] type: StructuredPrompt
  p_full['sys']['system'].value: You are a helpful assistant.
  p_full['usr']['query'].value: What is Python?


## 5. Deeper Nesting

You can nest prompts multiple levels deep:

In [5]:
# Build a 3-level nested structure
inst1 = "Be polite"
inst2 = "Be concise"
user = "Alice"

p_inst = prompt(t"Instructions: {inst1:i1}, {inst2:i2}")
p_user = prompt(t"User: {user}")
p_full = prompt(t"{p_inst:instructions}\n{p_user:user_info}")

print("Rendered:")
print(str(p_full))
print()

# Navigate deeply
print("Deep navigation:")
print(f"  First instruction: {p_full['instructions']['i1'].value}")
print(f"  Second instruction: {p_full['instructions']['i2'].value}")
print(f"  User: {p_full['user_info']['user'].value}")

Rendered:
Instructions: Be polite, Be concise
User: Alice

Deep navigation:
  First instruction: Be polite
  Second instruction: Be concise
  User: Alice


## 6. Provenance Export

Export full provenance or just values for logging/debugging:

In [6]:
import json

context = "User is Alice"
instructions = "Be concise"

p = prompt(t"Context: {context:ctx}. {instructions:inst}")

# Get just the values (simplified)
values = p.to_values()
print("Values:")
print(json.dumps(values, indent=2))
print()

# Get full provenance (all metadata)
provenance = p.to_provenance()
print("Full provenance:")
print(json.dumps(provenance, indent=2))

Values:
{
  "ctx": "User is Alice",
  "inst": "Be concise"
}

Full provenance:
{
  "strings": [
    "Context: ",
    ". ",
    ""
  ],
  "nodes": [
    {
      "key": "ctx",
      "expression": "context",
      "conversion": null,
      "format_spec": "ctx",
      "index": 0,
      "value": "User is Alice"
    },
    {
      "key": "inst",
      "expression": "instructions",
      "conversion": null,
      "format_spec": "inst",
      "index": 1,
      "value": "Be concise"
    }
  ]
}


## 7. Mapping Protocol

`StructuredPrompt` implements the Mapping protocol, so it works like a dictionary:

In [7]:
x = "X"
y = "Y"
z = "Z"

p = prompt(t"{x:a} {y:b} {z:c}")

# Dictionary operations
print(f"Length: {len(p)}")
print(f"Keys: {list(p.keys())}")
print(f"'a' in p: {'a' in p}")
print(f"'d' in p: {'d' in p}")
print()

# Iterate over key-value pairs
print("Items:")
for key, node in p.items():
    print(f"  {key}: {node.value}")

Length: 3
Keys: ['a', 'b', 'c']
'a' in p: True
'd' in p: False

Items:
  a: X
  b: Y
  c: Z


## 8. Template Introspection

Access the original template structure:

In [8]:
x = "X"
y = "Y"

p = prompt(t"before {x:a} middle {y:b} after")

# Static string segments
print(f"String segments: {p.strings}")
print()

# Interpolation nodes
print("Interpolations:")
for node in p.interpolations:
    print(f"  [{node.index}] key={node.key}, expr={node.expression}, value={node.value}")

String segments: ('before ', ' middle ', ' after')

Interpolations:
  [0] key=a, expr=x, value=X
  [1] key=b, expr=y, value=Y


## 9. Format Specs as Keys (Not Formatting)

By default, format specs are used as keys, not for formatting:

In [9]:
num = "42"

# "05d" looks like a format spec, but it's used as a key
p = prompt(t"{num:05d}")

print(f"Rendered (no formatting): {str(p)}")
print(f"Key: {list(p.keys())[0]}")
print()

# Optional: apply format specs for actual formatting
p2 = prompt(t"{num:>5}")
print(f"Without apply_format_spec: '{p2.render()}'")
print(f"With apply_format_spec: '{p2.render(apply_format_spec=True)}'")

Rendered (no formatting): 42
Key: 05d

Without apply_format_spec: '42'
With apply_format_spec: '   42'


## 10. Type Safety

Only strings and nested `StructuredPrompt`s are allowed as values:

In [10]:
# This works: string value
text = "hello"
p1 = prompt(t"{text}")
print(f"String value: {str(p1)}")

# This works: nested StructuredPrompt
inner = prompt(t"inner")
p2 = prompt(t"{inner}")
print(f"Nested prompt: {str(p2)}")
print()

# This fails: integer value
try:
    num = 42
    p3 = prompt(t"{num}")
except Exception as e:
    print(f"Error with int: {type(e).__name__}")
    print(f"  Message: {str(e)}")

String value: hello
Nested prompt: inner

Error with int: UnsupportedValueTypeError
  Message: Unsupported value type for interpolation 'num' (key='num'): expected str or StructuredPrompt, got int


## Summary

This demo covered the key features of `t-prompts`:

✅ **Basic usage**: Create prompts from t-strings with `prompt()`  
✅ **Provenance**: Access expression, key, conversion, format_spec, value  
✅ **Keying**: Format spec as key (if present), else expression  
✅ **Conversions**: Support for !s, !r, !a  
✅ **Composition**: Nest prompts to build complex structures  
✅ **Navigation**: Dict-like access with chaining (`p['a']['b']['c']`)  
✅ **Export**: `to_values()` and `to_provenance()` for JSON serialization  
✅ **Mapping protocol**: Works like a dictionary  
✅ **Introspection**: Access template strings and interpolations  
✅ **Type safety**: Only str and StructuredPrompt allowed  

**Use cases:**
- Building LLM prompts with full audit trails
- Composing complex prompts from reusable pieces
- Tracking which variables produced which prompt sections
- Debugging and testing prompt generation logic
- Logging structured prompt metadata for analysis